In [2]:
import torch
from torch import nn
from torch.nn import functional as F

print(torch.cuda.is_available())

True


In [3]:
class BasicBlock(nn.Module):

    def __init__(self, in_channels, out_channels, stride= 1):
        super().__init__()
        self.conv1 = nn.Conv2d(in_channels, out_channels, 3, stride= stride, padding= 1)
        self.conv2 = nn.Conv2d(out_channels, out_channels, 3, stride= 1, padding= 1)
        self.bn1 = nn.BatchNorm2d(out_channels)
        self.bn2 = nn.BatchNorm2d(out_channels)
        self.relu = nn.ReLU(inplace= True)

        self.downsample = nn.Sequential()
        
        if stride != 1:
            self.downsample = nn.Sequential(
                nn.Conv2d(in_channels, out_channels, 1, stride),
                nn.BatchNorm2d(out_channels))

    def forward(self, x):
        out = self.conv1(x)
        out = self.bn1(out)
        out = self.relu(out)

        out = self.conv2(out)
        out = self.bn2(out)

        out += self.downsample(x)

        out = self.relu(out)

        return out

class ResNet34(nn.Module):
    def __init__(self, num_classes) -> None:
        super().__init__()

        self.conv1 = nn.Sequential(nn.Conv2d(3, 64, 7, stride= 2, padding= 3),
            nn.BatchNorm2d(64), nn.MaxPool2d(3, stride= 2, padding= 1), nn.ReLU(inplace= True))

        self.layer0 = self._make_layer(64, 64, 3, 1)
        self.layer1 = self._make_layer(64, 128, 4, 2)
        self.layer2 = self._make_layer(128, 256, 6, 2)
        self.layer3 = self._make_layer(256, 512, 3, 2)

        self.avg_pool = nn.AvgPool2d(7)
        self.fc = nn.Linear(512, num_classes)

    def _make_layer(self, in_channels, out_channels, num_blocks, stride):
        
        layers = []
        layers.append(BasicBlock(in_channels, out_channels, stride))

        for i in range(num_blocks - 1):
            layers.append(BasicBlock(out_channels, out_channels, 1))

        return nn.Sequential(*layers)

    def forward(self, x):
        out = self.conv1(x)
        
        out = self.layer0(out)
        out = self.layer1(out)
        out = self.layer2(out)
        out = self.layer3(out)

        out = self.avg_pool(out)
        out = out.view(out.size(0), -1)
        out = self.fc(out)

        out = F.log_softmax(out, dim= 1)
        return out

In [4]:
model = ResNet34(450)
model.load_state_dict(torch.load('/kaggle/input/resnet34/birds_resnet34_model.pth'))

dummy_input = torch.randn(1, 3, 224, 224)

model.eval()
torch_out = model(dummy_input)

In [9]:
onnx_path = 'model.onnx'

torch.onnx.export(model,
                dummy_input,
                onnx_path,
                verbose=True,
                input_names = ['input'],   # the model's input names
                output_names = ['output'], # the model's output names
                dynamic_axes={'input' : {0 : 'batch_size'},    # variable length axes
                              'output' : {0 : 'batch_size'}})

graph(%input : Float(*, 3, 224, 224, strides=[150528, 50176, 224, 1], requires_grad=0, device=cpu),
      %fc.weight : Float(450, 512, strides=[512, 1], requires_grad=1, device=cpu),
      %fc.bias : Float(450, strides=[1], requires_grad=1, device=cpu),
      %390 : Float(64, 3, 7, 7, strides=[147, 49, 7, 1], requires_grad=0, device=cpu),
      %391 : Float(64, strides=[1], requires_grad=0, device=cpu),
      %393 : Float(64, 64, 3, 3, strides=[576, 9, 3, 1], requires_grad=0, device=cpu),
      %394 : Float(64, strides=[1], requires_grad=0, device=cpu),
      %396 : Float(64, 64, 3, 3, strides=[576, 9, 3, 1], requires_grad=0, device=cpu),
      %397 : Float(64, strides=[1], requires_grad=0, device=cpu),
      %399 : Float(64, 64, 3, 3, strides=[576, 9, 3, 1], requires_grad=0, device=cpu),
      %400 : Float(64, strides=[1], requires_grad=0, device=cpu),
      %402 : Float(64, 64, 3, 3, strides=[576, 9, 3, 1], requires_grad=0, device=cpu),
      %403 : Float(64, strides=[1], requires_gr

In [11]:
import onnx

onnx_model = onnx.load(onnx_path)
onnx.checker.check_model(onnx_model)

In [13]:
!pip install onnxruntime

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.5/4.5 MB 7.5 MB/s eta 0:00:0000:0100:010m
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.0/46.0 kB 3.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.8/86.8 kB 8.9 MB/s eta 0:00:00


In [14]:
import onnxruntime
import numpy as np

ort_session = onnxruntime.InferenceSession(onnx_path)

def to_numpy(tensor):
    return tensor.detach().cpu().numpy() if tensor.requires_grad else tensor.cpu().numpy()

ort_inputs = {ort_session.get_inputs()[0].name: to_numpy(dummy_input)}
ort_outs = ort_session.run(None, ort_inputs)

np.testing.assert_allclose(to_numpy(torch_out), ort_outs[0], rtol=1e-03, atol=1e-05)

print("Exported model has been tested with ONNXRuntime, and the result looks good!")

Exported model has been tested with ONNXRuntime, and the result looks good!
